In [22]:
import os
import xarray as xr
import geopandas as gpd
import pandas as pd
from datetime import datetime
!pip install rasterstats
from rasterstats import zonal_stats
import numpy as np
from shapely.geometry import Point, box
from rasterio.features import geometry_mask
import time
import xarray as xr
from shapely.geometry import mapping
import rioxarray
from datetime import datetime
from scipy.stats import genextreme
from concurrent.futures import ProcessPoolExecutor
import gcsfs  # For interacting with Google Cloud Storage
import tempfile
import shutil
import os
import time
import pandas as pd
from tempfile import TemporaryDirectory
from google.cloud import storage

# Functions

In [2]:
def calculate_pixel_weights_mSWEP(county_shapefile, precipitation_nc_path, output_path, gcs_bucket):
    # Set up Google Cloud Storage filesystem
    fs = gcsfs.GCSFileSystem()

    # Construct full GCS paths
    precipitation_nc_path = f'{gcs_bucket}/{precipitation_nc_path}'
    output_path = f'{gcs_bucket}/{output_path}'

    # Load the county shapefile locally
    counties = gpd.read_file(county_shapefile)

    # Load the precipitation NetCDF file from GCS
    with fs.open(precipitation_nc_path, 'rb') as f:
        ds = xr.open_dataset(f)

    # Ensure the precipitation data has a CRS
    if not ds.rio.crs:
        ds = ds.rio.write_crs(counties.crs, inplace=True)
        print("Assigned county shapefile CRS to precipitation data:", ds.rio.crs)

    precip = ds['precipitation']  # Adjust variable name if needed

    # Reproject counties to match precipitation data
    counties = counties.to_crs(precip.rio.crs)

    # Define the bounding box of the precipitation data
    precip_extent = box(
        precip['lon'].min().item(), precip['lat'].min().item(),
        precip['lon'].max().item(), precip['lat'].max().item()
    )

    # Clip counties to the extent of the precipitation data
    counties_clipped = counties[counties.intersects(precip_extent)]

    # Extract pixel centroids
    pixel_lon, pixel_lat = np.meshgrid(precip['lon'].values, precip['lat'].values)
    pixel_centroids = gpd.GeoDataFrame(
        {'geometry': [Point(x, y) for x, y in zip(pixel_lon.ravel(), pixel_lat.ravel())]},
        crs=precip.rio.crs
    )

    # Calculate weights for each pixel within each county
    weights = {}

    for county in counties_clipped.itertuples():
        # Create a mask for the current county
        mask = geometry_mask([county.geometry], transform=precip.rio.transform(), invert=True, out_shape=precip.shape[-2:])

        # Calculate the area of each pixel within the county
        pixel_areas = mask.astype(int)
        total_area = pixel_areas.sum()

        if total_area > 0:
            pixel_weights = pixel_areas / total_area
        else:
            # Find the closest pixel centroid to the county centroid if no pixels are contained
            county_centroid = county.geometry.centroid
            pixel_centroids['distance'] = pixel_centroids['geometry'].distance(county_centroid)
            closest_pixel = pixel_centroids.loc[pixel_centroids['distance'].idxmin()]

            # Identify the closest pixel's index in the precipitation grid
            closest_pixel_idx = (
                np.abs(precip['lat'] - closest_pixel.geometry.y).argmin(),
                np.abs(precip['lon'] - closest_pixel.geometry.x).argmin()
            )

            # Assign a weight of 1 to the closest pixel
            pixel_weights = np.zeros_like(pixel_areas)
            pixel_weights[closest_pixel_idx] = 1

        weights[county.GEOID] = pixel_weights

    # Save the weights dictionary to GCS
    with fs.open(output_path, 'wb') as f:
        np.save(f, weights)

    return weights

In [11]:
def process_mSWEP_precipitation(county_shapefile, precipitation_dir, weights, output_dir, gcs_bucket):
    # Set up Google Cloud Storage filesystem
    fs = gcsfs.GCSFileSystem()

    # Construct full GCS paths
    precipitation_dir_path = f'{gcs_bucket}/{precipitation_dir}'
    output_dir_path = f'{gcs_bucket}/{output_dir}'

    # Load the county shapefile locally
    counties = gpd.read_file(county_shapefile)

    # Ensure the output directory exists on GCS
    if not fs.exists(output_dir_path):
        fs.mkdirs(output_dir_path)

    # List all files in the precipitation directory on GCS
    precipitation_files = [f for f in fs.ls(precipitation_dir_path) if f.endswith('.nc')]

    # Start time for processing
    overall_start_time = time.time()

    # Iterate over each NetCDF file in the directory
    for year_file_path in sorted(precipitation_files):
        print(f"Processing file: {year_file_path}")

        # Extract the year from the filename (filename is structured as 'mswep_daily_precipitation_1950.nc')
        year_file_name = os.path.basename(year_file_path)
        year = year_file_name.split('_')[-1].split('.')[0]

        # Open the NetCDF file
        ds = xr.open_dataset(fs.open(year_file_path, 'rb'))

        # Load the precipitation variable
        precip = ds['precipitation']  # Ensure this matches the variable in your dataset
        year_records = []

        # Iterate over each county
        for county in counties.itertuples():
            geo_id = county.GEOID
            weight = weights.get(geo_id)

            if weight is not None:
                # Calculate the weighted average precipitation for each day
                daily_precip = (precip * weight).sum(dim=['lat', 'lon'])

                # Add results to the list for the current year
                for date, value in zip(ds['time'].values, daily_precip.values):
                    year_records.append({
                        'date': date,
                        'county': geo_id,
                        'precipitation': value
                    })

        # Convert the list of records for the year into a DataFrame
        year_df = pd.DataFrame.from_records(year_records)

        # Save the result for the year to a CSV file in GCS
        output_csv_path = f"{output_dir_path}/daily_precip_by_county_{year}.csv"
        with fs.open(output_csv_path, 'w') as f:
            year_df.to_csv(f, index=False)

        # Calculate elapsed time for this year
        year_elapsed_time = time.time() - overall_start_time
        print(f"Saved results for {year} to {output_csv_path} in {year_elapsed_time:.2f} seconds")
        overall_start_time = time.time()

In [16]:
def rechunk_precipitation_data_by_county_gcs(years, input_path, output_path, gcs_bucket):
    # Set up Google Cloud Storage filesystem
    fs = gcsfs.GCSFileSystem()

    # Construct full GCS paths
    input_path_gcs = f"{gcs_bucket}/{input_path}"
    output_path_gcs = f"{gcs_bucket}/{output_path}"

    with tempfile.TemporaryDirectory() as temp_dir:
        print(f"Temporary directory created at {temp_dir}")

        # Step 1: Process each year and store county-specific files in a separate folder
        yearly_temp_dirs = []

        for year in years:
            start_time = time.time()
            print(f"Processing year {year}...")

            yearly_temp_dir = os.path.join(temp_dir, f"year_{year}")
            os.makedirs(yearly_temp_dir, exist_ok=True)
            yearly_temp_dirs.append(yearly_temp_dir)

            # Read the year's data from GCS
            input_file_path = f"{input_path_gcs}/{year}_MSWEP.csv"
            with fs.open(input_file_path, 'r') as f:
                yearly_data = pd.read_csv(f)

            # Ensure FIPS is 5 characters and dates are in datetime format
            yearly_data['county'] = yearly_data['county'].apply(lambda x: f"{int(x):05d}")
            yearly_data['date'] = pd.to_datetime(yearly_data['date'])

            # Write data for each county into the year's temp folder
            for county, group in yearly_data.groupby('county'):
                county_file_path = os.path.join(yearly_temp_dir, f"{county}_MSWEP.csv")
                group.to_csv(county_file_path, index=False)

            end_time = time.time()
            print(f"Year {year} processed in {end_time - start_time:.2f} seconds.")

        # Step 2: Merge yearly county files and upload to GCS
        print("Merging yearly files and uploading to GCS...")
        county_files = {}

        for yearly_temp_dir in yearly_temp_dirs:
            for county_file in os.listdir(yearly_temp_dir):
                county = county_file.split("_")[0]
                yearly_file_path = os.path.join(yearly_temp_dir, county_file)

                if county not in county_files:
                    county_files[county] = pd.read_csv(yearly_file_path)
                else:
                    new_data = pd.read_csv(yearly_file_path)
                    county_files[county] = pd.concat([county_files[county], new_data]).drop_duplicates()

        # Write final county files and upload to GCS
        for county, df in county_files.items():
            local_file_path = os.path.join(temp_dir, f"{county}_MSWEP.csv")
            df.to_csv(local_file_path, index=False)

            # Upload to GCS
            output_file_path = f"{output_path_gcs}/{county}_MSWEP.csv"
            with fs.open(output_file_path, 'w') as f:
                f.write(open(local_file_path, 'r').read())

        print("Upload complete. Temporary directory cleaned up.")

In [46]:
def download_from_gcs(bucket_name, source_folder, local_folder):
    """Download all files from a GCS bucket folder to a local folder."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=source_folder)
    
    os.makedirs(local_folder, exist_ok=True)
    for blob in blobs:
        if blob.name.endswith('.csv'):
            destination = os.path.join(local_folder, os.path.basename(blob.name))
            blob.download_to_filename(destination)

def process_precipitation_data_mswep(file_path):
    county = os.path.basename(file_path).split('_')[0]
    start_time = time.time()

    # Load and sort data by date
    df = pd.read_csv(file_path, parse_dates=['date'])
    df.sort_values('date', inplace=True)

    # Rename the 'precipitation' column to 'MSWEP_precipitation'
    df.rename(columns={'precipitation': 'MSWEP_precipitation'}, inplace=True)

    # Calculate the specified metrics within a 30-day rolling window
    df['MSWEP_precipitation_30d_sum'] = df['MSWEP_precipitation'].rolling(window=30, min_periods=1).sum()
    df['MSWEP_precipitation_30d_max_1d'] = df['MSWEP_precipitation'].rolling(window=30, min_periods=1).max()
    df['MSWEP_precipitation_30d_max_3d'] = (
        df['MSWEP_precipitation'].rolling(window=3, min_periods=1).sum()
        .rolling(window=30, min_periods=1).max()
    )
    df['MSWEP_precipitation_30d_max_5d'] = (
        df['MSWEP_precipitation'].rolling(window=5, min_periods=1).sum()
        .rolling(window=30, min_periods=1).max()
    )
    df['MSWEP_precipitation_30d_max_7d'] = (
        df['MSWEP_precipitation'].rolling(window=7, min_periods=1).sum()
        .rolling(window=30, min_periods=1).max()
    )
    df['MSWEP_precipitation_30d_max_14d'] = (
        df['MSWEP_precipitation'].rolling(window=14, min_periods=1).sum()
        .rolling(window=30, min_periods=1).max()
    )

    # Extract year from the date column for annual aggregation
    df['year'] = df['date'].dt.year

    # Create aggregation dictionary for annual maxima
    aggregation_dict = {
        'MSWEP_precipitation_30d_sum': 'max',
        'MSWEP_precipitation_30d_max_1d': 'max',
        'MSWEP_precipitation_30d_max_3d': 'max',
        'MSWEP_precipitation_30d_max_5d': 'max',
        'MSWEP_precipitation_30d_max_7d': 'max',
        'MSWEP_precipitation_30d_max_14d': 'max',
        'MSWEP_precipitation': 'max'  # daily max of base precipitation
    }

    # Calculate annual maxima
    annual_maxima = df.groupby('year').agg(aggregation_dict).reset_index()

    # Fit GEV distribution and calculate percentiles for each column
    columns_to_fit = list(aggregation_dict.keys())
    for column in columns_to_fit:
        annual_max_distribution = annual_maxima[column].dropna().values
        if len(annual_max_distribution) > 0:
            shape, loc, scale = genextreme.fit(annual_max_distribution)
            df[f'{column}_percentile_modeled'] = genextreme.cdf(df[column], shape, loc, scale) * 100

    # Save the processed data and annual maxima
    processed_folder = 'MSWEP_Daily_Precip_Processed_County'
    os.makedirs(processed_folder, exist_ok=True)
    df.to_csv(f'{processed_folder}/{county}_precip_processed.csv', index=False)

    annual_max_folder = 'MSWEP_Ann_Max_Precip_County'
    os.makedirs(annual_max_folder, exist_ok=True)
    annual_maxima.to_csv(f'{annual_max_folder}/Ann_max_precip_county_{county}.csv', index=False)

    end_time = time.time()
    print(f"County {county} processed in {end_time - start_time:.2f} seconds.")

    return df, annual_maxima

def process_all_precipitation_data_mswep(directory):
    csv_files = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.csv')]

    # Using ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor() as executor:
        executor.map(process_precipitation_data_mswep, csv_files)

In [51]:
def merge_claims_with_precip(claims_folder, precip_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Process each claims file in the Claims_By_County folder
    for claims_file in os.listdir(claims_folder):
        if claims_file.endswith('_claims_merged.csv'):
            # Extract county name from file name
            county = claims_file.split('_')[0]
            
            # Construct file paths
            claims_path = os.path.join(claims_folder, claims_file)
            precip_path = os.path.join(precip_folder, f'{county}_precip_processed.csv')
            
            # Check if the corresponding precipitation file exists
            if not os.path.exists(precip_path):
                print(f"No precipitation file found for {county}. Skipping.")
                continue
            
            # Read the claims and precipitation data
            claims_df = pd.read_csv(claims_path)
            precip_df = pd.read_csv(precip_path)
            
            # Convert 'dateOfLoss' and 'date' to datetime if not already
            claims_df['dateOfLoss'] = pd.to_datetime(claims_df['dateOfLoss'])
            precip_df['date'] = pd.to_datetime(precip_df['date'])
            
            # Align the timezones (convert to UTC for consistency)
            claims_df['dateOfLoss'] = claims_df['dateOfLoss'].dt.tz_localize(None)
            precip_df['date'] = precip_df['date'].dt.tz_localize(None)
            
            # Merge on the date
            merged_df = pd.merge(claims_df, precip_df, left_on='dateOfLoss', right_on='date', how='left')
            
            # Drop the extra 'date' and 'county' columns after merge
            merged_df.drop(columns=['date', 'county', 'year'], inplace=True, errors='ignore')
            
            # Output the merged DataFrame to the new folder
            output_path = os.path.join(output_folder, f'{county}_claims_merged.csv')
            merged_df.to_csv(output_path, index=False)
            
            print(f"County {county} claims merged and saved to {output_path}.")

In [55]:
def concatenate_processed_claims(output_file="Processed_Claims_ERA5.csv", processed_path="Output_Claims_ERA5"):
    # List all files in the processed claims directory
    files = [f for f in os.listdir(processed_path) if f.endswith('_claims_merged.csv')]
    
    # Initialize an empty list to store DataFrames
    df_list = []

    # Loop through the files and read them into DataFrames
    for file in files:
        file_path = os.path.join(processed_path, file)
        df = pd.read_csv(file_path)
        df_list.append(df)
        print(f"Loaded {file}")

    # Concatenate all DataFrames into one
    combined_df = pd.concat(df_list, ignore_index=True)

    # Save the combined DataFrame to a single CSV file
    combined_df.to_csv(output_file, index=False)
    print(f"All county files concatenated into {output_file}")

In [56]:
def push_to_cloud_and_conditional_remove(local_paths, remove_flags, bucket_name, target_folder):
    """
    Push files or folders to the cloud and conditionally remove them from the local system.

    Parameters:
        local_paths (list): List of file or folder paths to upload.
        remove_flags (list): List of boolean values indicating whether to remove each item after upload.
        bucket_name (str): The name of the Google Cloud Storage bucket.
        target_folder (str): The destination folder path in the bucket.
    """
    # Initialize the client
    client = storage.Client()

    if len(local_paths) != len(remove_flags):
        raise ValueError("The number of paths and remove flags must be the same.")

    for local_path, remove_flag in zip(local_paths, remove_flags):
        # Check if the local path exists
        if not os.path.exists(local_path):
            print(f"Local path does not exist: {local_path}")
            continue

        # Access the bucket
        try:
            bucket = client.bucket(bucket_name)
            print(f"Bucket accessed: {bucket_name}")
        except Exception as e:
            print(f"Failed to access bucket: {e}")
            raise

        # Check if the path is a file or folder
        try:
            if os.path.isfile(local_path):
                # Upload the single file
                relative_path = os.path.basename(local_path)
                blob_path = f"{target_folder}/{relative_path}"
                blob = bucket.blob(blob_path)
                blob.upload_from_filename(local_path)
                print(f"Uploaded {local_path} to gs://{bucket_name}/{blob_path}")

                # Remove the local file if remove_flag is True
                if remove_flag:
                    os.remove(local_path)
                    print(f"Removed local file: {local_path}")

            elif os.path.isdir(local_path):
                # Use the folder name as a subfolder in the cloud
                base_folder_name = os.path.basename(local_path)
                folder_target_path = f"{target_folder}/{base_folder_name}"

                # Walk through the folder and upload each file
                for root, _, files in os.walk(local_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        # Only include the file name in the cloud path
                        relative_path = os.path.basename(file)
                        blob_path = f"{folder_target_path}/{relative_path}"

                        blob = bucket.blob(blob_path)
                        blob.upload_from_filename(file_path)
                        print(f"Uploaded {file_path} to gs://{bucket_name}/{blob_path}")

                # Remove the local folder and its contents if remove_flag is True
                if remove_flag:
                    shutil.rmtree(local_path)
                    print(f"Removed local folder and its contents: {local_path}")

            else:
                print(f"The path is neither a file nor a folder: {local_path}")
        except Exception as e:
            print(f"Error during file/folder upload: {e}")
            raise

# Input Files

In [5]:
# Define your bucket and paths
bucket = 'gs://leap-persistent/adamnayak/flood-insurance'
county_shapefile = '../Local_Data/Geospatial/tl_2019_us_county.shp'
precipitation_nc_path = 'MSWEP/Daily_Annual/MSWEP_1979.nc'
weights_output_path = 'MSWEP/MSWEP_county_pixel_weights.npy'

# Calculate County Weights

In [6]:
# Call the function
weights = calculate_pixel_weights_mSWEP(
    county_shapefile=county_shapefile,
    precipitation_nc_path=precipitation_nc_path,
    output_path=weights_output_path,
    gcs_bucket=bucket
)

Assigned county shapefile CRS to precipitation data: EPSG:4269


/tmp/ipykernel_4102/1773105646.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pixel_centroids['distance'] = pixel_centroids['geometry'].distance(county_centroid)
/tmp/ipykernel_4102/1773105646.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pixel_centroids['distance'] = pixel_centroids['geometry'].distance(county_centroid)
/tmp/ipykernel_4102/1773105646.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pixel_centroids['distance'] = pixel_centroids['geometry'].distance(county_centroid)
/tmp/ipykernel_4102/1773105646.py:58: UserWarning: Geometry is in a geogr

# Aggregate Daily Precipitation

In [9]:
# Define your paths
precipitation_dir = 'MSWEP/Daily_Annual/'
output_dir = 'MSWEP/County_Daily_Precip_By_Year'

In [12]:
# Call the function
process_mSWEP_precipitation(
    county_shapefile=county_shapefile,
    precipitation_dir=precipitation_dir,
    weights=weights,
    output_dir=output_dir,
    gcs_bucket=bucket
)

Processing file: leap-persistent/adamnayak/flood-insurance/MSWEP/Daily_Annual/MSWEP_1979.nc


KeyboardInterrupt: 

# Rechunk to County

In [13]:
# Define GCS bucket and paths
input_path = 'MSWEP/County_Daily_Precip_By_Year'
output_path = 'MSWEP/County_Daily_Precip_By_County'

# Define the range of years to process
precip_years = range(1979, 2021)

In [17]:
rechunk_precipitation_data_by_county_gcs(precip_years, 
                                         input_path, 
                                         output_path, 
                                         bucket)

Temporary directory created at /tmp/tmp3ip9v7tr
Processing year 1979...
Year 1979 processed in 5.76 seconds.
Processing year 1980...
Year 1980 processed in 5.25 seconds.
Processing year 1981...
Year 1981 processed in 5.49 seconds.
Processing year 1982...
Year 1982 processed in 5.47 seconds.
Processing year 1983...
Year 1983 processed in 5.23 seconds.
Processing year 1984...
Year 1984 processed in 5.58 seconds.
Processing year 1985...
Year 1985 processed in 5.64 seconds.
Processing year 1986...
Year 1986 processed in 5.38 seconds.
Processing year 1987...


KeyboardInterrupt: 

# Calculate County Precip Stats of Interest

In [23]:
bucket_name = 'leap-persistent'
source_folder = 'adamnayak/flood-insurance/MSWEP/County_Daily_Precip_By_County'
local_folder = 'MSWEP_County_Precip_Local'

In [24]:
# Download data from GCS
download_from_gcs(bucket_name, source_folder, local_folder)

In [47]:
# Process all precipitation data
process_all_precipitation_data_mswep(local_folder)

County 53075 processed in 1.03 seconds.
County 48375 processed in 1.22 seconds.
County 50013 processed in 1.33 seconds.
County 08005 processed in 1.34 seconds.
County 37181 processed in 1.36 seconds.
County 40065 processed in 1.43 seconds.
County 13195 processed in 1.47 seconds.County 20199 processed in 1.47 seconds.County 51685 processed in 1.46 seconds.


County 20171 processed in 1.40 seconds.County 39167 processed in 1.43 seconds.

County 51111 processed in 1.55 seconds.County 47009 processed in 1.55 seconds.

County 21219 processed in 1.59 seconds.
County 47077 processed in 1.58 seconds.
County 51595 processed in 1.61 seconds.
County 49037 processed in 1.06 seconds.
County 19113 processed in 1.46 seconds.
County 17063 processed in 1.35 seconds.County 13279 processed in 1.27 seconds.

County 27153 processed in 1.30 seconds.
County 54011 processed in 1.13 seconds.
County 28033 processed in 1.32 seconds.
County 13289 processed in 1.32 seconds.
County 12001 processed in 1.32 seconds.


# Merge with ERA5 Processed Data

In [48]:
# Define folders for analysis
claims_folder = 'ERA5_Merged_Claims'
precip_folder = 'MSWEP_Daily_Precip_Processed_County'
output_folder = 'MSWEP_ERA5_Merged_Claims'

In [52]:
# Merge claims with precipitation data
merge_claims_with_precip(claims_folder=claims_folder, 
                         precip_folder=precip_folder, 
                         output_folder=output_folder)

County 17189 claims merged and saved to MSWEP_ERA5_Merged_Claims/17189_claims_merged.csv.
County 05119 claims merged and saved to MSWEP_ERA5_Merged_Claims/05119_claims_merged.csv.
County 55105 claims merged and saved to MSWEP_ERA5_Merged_Claims/55105_claims_merged.csv.
County 22005 claims merged and saved to MSWEP_ERA5_Merged_Claims/22005_claims_merged.csv.
County 38045 claims merged and saved to MSWEP_ERA5_Merged_Claims/38045_claims_merged.csv.
County 53065 claims merged and saved to MSWEP_ERA5_Merged_Claims/53065_claims_merged.csv.
County 35027 claims merged and saved to MSWEP_ERA5_Merged_Claims/35027_claims_merged.csv.
County 21071 claims merged and saved to MSWEP_ERA5_Merged_Claims/21071_claims_merged.csv.
County 26009 claims merged and saved to MSWEP_ERA5_Merged_Claims/26009_claims_merged.csv.
County 38051 claims merged and saved to MSWEP_ERA5_Merged_Claims/38051_claims_merged.csv.
County 48221 claims merged and saved to MSWEP_ERA5_Merged_Claims/48221_claims_merged.csv.
County 040

/tmp/ipykernel_4102/3508933425.py:22: DtypeWarning: Columns (42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv(claims_path)


County 34025 claims merged and saved to MSWEP_ERA5_Merged_Claims/34025_claims_merged.csv.
County 30025 claims merged and saved to MSWEP_ERA5_Merged_Claims/30025_claims_merged.csv.
County 48073 claims merged and saved to MSWEP_ERA5_Merged_Claims/48073_claims_merged.csv.
County 06113 claims merged and saved to MSWEP_ERA5_Merged_Claims/06113_claims_merged.csv.
County 48093 claims merged and saved to MSWEP_ERA5_Merged_Claims/48093_claims_merged.csv.
County 54107 claims merged and saved to MSWEP_ERA5_Merged_Claims/54107_claims_merged.csv.
County 48193 claims merged and saved to MSWEP_ERA5_Merged_Claims/48193_claims_merged.csv.
County 42059 claims merged and saved to MSWEP_ERA5_Merged_Claims/42059_claims_merged.csv.
County 29510 claims merged and saved to MSWEP_ERA5_Merged_Claims/29510_claims_merged.csv.
County 20005 claims merged and saved to MSWEP_ERA5_Merged_Claims/20005_claims_merged.csv.
County 39007 claims merged and saved to MSWEP_ERA5_Merged_Claims/39007_claims_merged.csv.
County 291

/tmp/ipykernel_4102/3508933425.py:22: DtypeWarning: Columns (7,42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv(claims_path)


County 22109 claims merged and saved to MSWEP_ERA5_Merged_Claims/22109_claims_merged.csv.
County 36069 claims merged and saved to MSWEP_ERA5_Merged_Claims/36069_claims_merged.csv.
County 36079 claims merged and saved to MSWEP_ERA5_Merged_Claims/36079_claims_merged.csv.
County 20175 claims merged and saved to MSWEP_ERA5_Merged_Claims/20175_claims_merged.csv.
County 13089 claims merged and saved to MSWEP_ERA5_Merged_Claims/13089_claims_merged.csv.
County 13033 claims merged and saved to MSWEP_ERA5_Merged_Claims/13033_claims_merged.csv.
County 48149 claims merged and saved to MSWEP_ERA5_Merged_Claims/48149_claims_merged.csv.
County 29045 claims merged and saved to MSWEP_ERA5_Merged_Claims/29045_claims_merged.csv.
County 24510 claims merged and saved to MSWEP_ERA5_Merged_Claims/24510_claims_merged.csv.
County 19103 claims merged and saved to MSWEP_ERA5_Merged_Claims/19103_claims_merged.csv.
County 37085 claims merged and saved to MSWEP_ERA5_Merged_Claims/37085_claims_merged.csv.
County 230

...

County 34009 claims merged and saved to MSWEP_ERA5_Merged_Claims/34009_claims_merged.csv.
County 31037 claims merged and saved to MSWEP_ERA5_Merged_Claims/31037_claims_merged.csv.
County 20115 claims merged and saved to MSWEP_ERA5_Merged_Claims/20115_claims_merged.csv.
County 21163 claims merged and saved to MSWEP_ERA5_Merged_Claims/21163_claims_merged.csv.
County 13161 claims merged and saved to MSWEP_ERA5_Merged_Claims/13161_claims_merged.csv.
County 39059 claims merged and saved to MSWEP_ERA5_Merged_Claims/39059_claims_merged.csv.
County 38049 claims merged and saved to MSWEP_ERA5_Merged_Claims/38049_claims_merged.csv.
County 25025 claims merged and saved to MSWEP_ERA5_Merged_Claims/25025_claims_merged.csv.
County 27009 claims merged and saved to MSWEP_ERA5_Merged_Claims/27009_claims_merged.csv.
County 40121 claims merged and saved to MSWEP_ERA5_Merged_Claims/40121_claims_merged.csv.
County 26021 claims merged and saved to MSWEP_ERA5_Merged_Claims/26021_claims_merged.csv.
County 371

/tmp/ipykernel_4102/3508933425.py:22: DtypeWarning: Columns (7,42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv(claims_path)


County 37129 claims merged and saved to MSWEP_ERA5_Merged_Claims/37129_claims_merged.csv.
County 27129 claims merged and saved to MSWEP_ERA5_Merged_Claims/27129_claims_merged.csv.
County 41013 claims merged and saved to MSWEP_ERA5_Merged_Claims/41013_claims_merged.csv.
County 42123 claims merged and saved to MSWEP_ERA5_Merged_Claims/42123_claims_merged.csv.


/tmp/ipykernel_4102/3508933425.py:22: DtypeWarning: Columns (35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv(claims_path)


County 29189 claims merged and saved to MSWEP_ERA5_Merged_Claims/29189_claims_merged.csv.
County 18015 claims merged and saved to MSWEP_ERA5_Merged_Claims/18015_claims_merged.csv.
County 47187 claims merged and saved to MSWEP_ERA5_Merged_Claims/47187_claims_merged.csv.
County 40151 claims merged and saved to MSWEP_ERA5_Merged_Claims/40151_claims_merged.csv.
County 22093 claims merged and saved to MSWEP_ERA5_Merged_Claims/22093_claims_merged.csv.
County 06017 claims merged and saved to MSWEP_ERA5_Merged_Claims/06017_claims_merged.csv.
County 13097 claims merged and saved to MSWEP_ERA5_Merged_Claims/13097_claims_merged.csv.
County 17039 claims merged and saved to MSWEP_ERA5_Merged_Claims/17039_claims_merged.csv.
County 19091 claims merged and saved to MSWEP_ERA5_Merged_Claims/19091_claims_merged.csv.
County 51101 claims merged and saved to MSWEP_ERA5_Merged_Claims/51101_claims_merged.csv.
County 48471 claims merged and saved to MSWEP_ERA5_Merged_Claims/48471_claims_merged.csv.
County 200

# Concatenate Claims with Precip

In [57]:
processed_path="MSWEP_ERA5_Merged_Claims"
output_file="MSWEP_ERA5_Processed_Claims.csv"

In [58]:
concatenate_processed_claims(output_file=output_file, 
                                 processed_path=processed_path)

Loaded 17189_claims_merged.csv
Loaded 05119_claims_merged.csv
Loaded 55105_claims_merged.csv
Loaded 22005_claims_merged.csv
Loaded 38045_claims_merged.csv
Loaded 53065_claims_merged.csv
Loaded 35027_claims_merged.csv
Loaded 21071_claims_merged.csv
Loaded 26009_claims_merged.csv
Loaded 38051_claims_merged.csv
Loaded 48221_claims_merged.csv
Loaded 04005_claims_merged.csv
Loaded 29081_claims_merged.csv
Loaded 37007_claims_merged.csv
Loaded 21085_claims_merged.csv
Loaded 01121_claims_merged.csv
Loaded 48427_claims_merged.csv
Loaded 19037_claims_merged.csv
Loaded 56017_claims_merged.csv
Loaded 12129_claims_merged.csv
Loaded 46115_claims_merged.csv
Loaded 48441_claims_merged.csv
Loaded 24037_claims_merged.csv
Loaded 48231_claims_merged.csv
Loaded 27049_claims_merged.csv
Loaded 19009_claims_merged.csv
Loaded 17035_claims_merged.csv
Loaded 19069_claims_merged.csv
Loaded 40047_claims_merged.csv
Loaded 37177_claims_merged.csv
Loaded 26151_claims_merged.csv
Loaded 37077_claims_merged.csv
Loaded 3

/tmp/ipykernel_4102/91679860.py:11: DtypeWarning: Columns (42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded 34025_claims_merged.csv
Loaded 30025_claims_merged.csv
Loaded 48073_claims_merged.csv
Loaded 06113_claims_merged.csv
Loaded 48093_claims_merged.csv
Loaded 54107_claims_merged.csv
Loaded 48193_claims_merged.csv
Loaded 42059_claims_merged.csv
Loaded 29510_claims_merged.csv
Loaded 20005_claims_merged.csv
Loaded 39007_claims_merged.csv
Loaded 29107_claims_merged.csv
Loaded 51037_claims_merged.csv
Loaded 21153_claims_merged.csv
Loaded 54009_claims_merged.csv
Loaded 19181_claims_merged.csv
Loaded 05073_claims_merged.csv
Loaded 42079_claims_merged.csv
Loaded 21189_claims_merged.csv
Loaded 31041_claims_merged.csv
Loaded 51760_claims_merged.csv
Loaded 42107_claims_merged.csv
Loaded 45083_claims_merged.csv
Loaded 48163_claims_merged.csv
Loaded 51013_claims_merged.csv
Loaded 18035_claims_merged.csv
Loaded 01105_claims_merged.csv
Loaded 48161_claims_merged.csv
Loaded 18117_claims_merged.csv
Loaded 22111_claims_merged.csv
Loaded 33015_claims_merged.csv
Loaded 21233_claims_merged.csv
Loaded 1

/tmp/ipykernel_4102/91679860.py:11: DtypeWarning: Columns (7,42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded 22109_claims_merged.csv
Loaded 36069_claims_merged.csv
Loaded 36079_claims_merged.csv
Loaded 20175_claims_merged.csv
Loaded 13089_claims_merged.csv
Loaded 13033_claims_merged.csv
Loaded 48149_claims_merged.csv
Loaded 29045_claims_merged.csv
Loaded 24510_claims_merged.csv
Loaded 19103_claims_merged.csv
Loaded 37085_claims_merged.csv
Loaded 23025_claims_merged.csv
Loaded 33003_claims_merged.csv
Loaded 48109_claims_merged.csv
Loaded 17187_claims_merged.csv
Loaded 51127_claims_merged.csv
Loaded 55043_claims_merged.csv
Loaded 55139_claims_merged.csv
Loaded 28005_claims_merged.csv
Loaded 51049_claims_merged.csv
Loaded 16011_claims_merged.csv
Loaded 28049_claims_merged.csv
Loaded 12035_claims_merged.csv
Loaded 47031_claims_merged.csv
Loaded 19125_claims_merged.csv
Loaded 06053_claims_merged.csv
Loaded 13107_claims_merged.csv
Loaded 48253_claims_merged.csv
Loaded 18005_claims_merged.csv
Loaded 51177_claims_merged.csv
Loaded 27077_claims_merged.csv
Loaded 30033_claims_merged.csv
Loaded 5

...

Loaded 48157_claims_merged.csv
Loaded 36121_claims_merged.csv
Loaded 12105_claims_merged.csv
Loaded 23001_claims_merged.csv
Loaded 51087_claims_merged.csv
Loaded 35045_claims_merged.csv
Loaded 17065_claims_merged.csv
Loaded 21175_claims_merged.csv
Loaded 38069_claims_merged.csv
Loaded 37065_claims_merged.csv
Loaded 53015_claims_merged.csv
Loaded 12055_claims_merged.csv
Loaded 54047_claims_merged.csv
Loaded 48425_claims_merged.csv
Loaded 55071_claims_merged.csv
Loaded 27109_claims_merged.csv
Loaded 46015_claims_merged.csv
Loaded 48327_claims_merged.csv
Loaded 27111_claims_merged.csv
Loaded 47045_claims_merged.csv
Loaded 21179_claims_merged.csv
Loaded 51171_claims_merged.csv


/tmp/ipykernel_4102/91679860.py:11: DtypeWarning: Columns (7,35,42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded 34009_claims_merged.csv
Loaded 31037_claims_merged.csv
Loaded 20115_claims_merged.csv
Loaded 21163_claims_merged.csv
Loaded 13161_claims_merged.csv
Loaded 39059_claims_merged.csv
Loaded 38049_claims_merged.csv
Loaded 25025_claims_merged.csv
Loaded 27009_claims_merged.csv
Loaded 40121_claims_merged.csv
Loaded 26021_claims_merged.csv
Loaded 37173_claims_merged.csv
Loaded 27167_claims_merged.csv
Loaded 48007_claims_merged.csv
Loaded 48305_claims_merged.csv
Loaded 20099_claims_merged.csv
Loaded 33017_claims_merged.csv
Loaded 37119_claims_merged.csv
Loaded 42103_claims_merged.csv
Loaded 37041_claims_merged.csv
Loaded 18165_claims_merged.csv
Loaded 54101_claims_merged.csv
Loaded 49027_claims_merged.csv
Loaded 13053_claims_merged.csv
Loaded 01039_claims_merged.csv
Loaded 19147_claims_merged.csv
Loaded 38101_claims_merged.csv
Loaded 19111_claims_merged.csv
Loaded 37129_claims_merged.csv
Loaded 27129_claims_merged.csv
Loaded 41013_claims_merged.csv
Loaded 42123_claims_merged.csv


/tmp/ipykernel_4102/91679860.py:11: DtypeWarning: Columns (7,42,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_4102/91679860.py:11: DtypeWarning: Columns (35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded 29189_claims_merged.csv
Loaded 18015_claims_merged.csv
Loaded 47187_claims_merged.csv
Loaded 40151_claims_merged.csv
Loaded 22093_claims_merged.csv
Loaded 06017_claims_merged.csv
Loaded 13097_claims_merged.csv
Loaded 17039_claims_merged.csv
Loaded 19091_claims_merged.csv
Loaded 51101_claims_merged.csv
Loaded 48471_claims_merged.csv
Loaded 20077_claims_merged.csv
Loaded 17177_claims_merged.csv
Loaded 27081_claims_merged.csv
Loaded 48289_claims_merged.csv
Loaded 56041_claims_merged.csv
Loaded 29186_claims_merged.csv
Loaded 39109_claims_merged.csv
Loaded 16085_claims_merged.csv
Loaded 13147_claims_merged.csv
Loaded 55061_claims_merged.csv
Loaded 01015_claims_merged.csv
Loaded 08073_claims_merged.csv
Loaded 37091_claims_merged.csv
Loaded 35007_claims_merged.csv
All county files concatenated into MSWEP_ERA5_Processed_Claims.csv


# Clean Up

In [59]:
# Define bucket and folder paths
bucket_name = "leap-persistent"
target_folder = "adamnayak/flood-insurance/MSWEP"  # Updated path

In [60]:
local_paths = ['MSWEP_Ann_Max_Precip_County', 'MSWEP_Daily_Precip_Processed_County', 'MSWEP_ERA5_Merged_Claims', 'MSWEP_ERA5_Processed_Claims.csv']
remove_flags = [True, True, False, False]  # Specify which items to remove after upload

push_to_cloud_and_conditional_remove(local_paths, remove_flags, bucket_name, target_folder)

Bucket accessed: leap-persistent
Uploaded MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_31049.csv to gs://leap-persistent/adamnayak/flood-insurance/MSWEP/MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_31049.csv
Uploaded MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_10003.csv to gs://leap-persistent/adamnayak/flood-insurance/MSWEP/MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_10003.csv
Uploaded MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_48415.csv to gs://leap-persistent/adamnayak/flood-insurance/MSWEP/MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_48415.csv
Uploaded MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_50013.csv to gs://leap-persistent/adamnayak/flood-insurance/MSWEP/MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_50013.csv
Uploaded MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_13187.csv to gs://leap-persistent/adamnayak/flood-insurance/MSWEP/MSWEP_Ann_Max_Precip_County/Ann_max_precip_county_13187.csv
Uploaded MSWEP_Ann_Max_Precip_County/Ann_m

In [61]:
!rm -r MSWEP_County_Precip_Local
!rm -r ERA5_Merged_Claims
# Note that we want to keep MSWEP_ERA5_Merged_Claims folder until all datasets are merged to the claims